### Data Aggregations
(Spark)

Agregações prontas para serem consumidas pelo Dash Plotly, ou seja, devem estar em CSV.

* Inputs:
    * /datasets/prepared/df_covid_complete/ (parquet)
* Outputs:
    * /aggregations/df_new_month_br.csv
    * /aggregations/df_new_day_br.csv
    * /aggregations/df_new_weekday_br.csv
    * /aggregations/df_mvg_avg_br.csv
    * /aggregations/df_let_br.csv
    * /aggregations/df_new_weekday_reg.csv
    * /aggregations/df_let_reg.csv
    * /aggregations/df_new_month_uf.csv
    * /aggregations/df_mvg_avg_uf.csv
    * /aggregations/df_let_uf.csv
    * /aggregations/df_new_month_munic.csv
    * /aggregations/df_mvg_avg_munic.csv
    * /aggregations/df_let_munic.csv

In [1]:
# Spark
import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.types as t
from pyspark.sql.window import Window

spark = SparkSession \
            .builder \
            .config("spark.sql.broadcastTimeout", "360000") \
            .config('spark.sql.execution.arrow.enabled', 'false') \
            .config("spark.driver.memory", '14G') \
            .config("spark.executor.memory", '14G') \
            .config("spark.driver.maxResultSize", '4G') \
            .getOrCreate()

In [2]:
import pandas as pd
from itertools import chain

### 0. Constantes

In [3]:
main_path = 'D:/data_dash_covid/datasets/'

In [4]:
# Média móvel de 14 dias (período incubação)
dias_mm = 13

In [5]:
# Última data disponível do dado
data_max = '2020-10-24'

### 1. Read data

In [6]:
df = spark.read.parquet(main_path+'prepared/df_covid_complete/')

### 2. Agregações Brasil

* Novos Casos x Mês x Brasil
* Novas Mortes x Mês x Brasil
* Novos Casos x Dia x Brasil
* Novas Mortes x Dia x Brasil
* Novos Casos x Dia Semana x Brasil
* Novas Mortes x Dia Semana x Brasil
* Média Móvel Casos x Dia x Brasil
* Média Móvel Mortes x Dia x Brasil
* Letalidade x Último Dia x Brasil

#### 2.1. Novos Casos e Mortes x Mês x Brasil

In [7]:
df_new_month_br = df.withColumn('País',f.lit('Brasil'))\
                     .groupBy('Mês','Mês_Nome','País')\
                     .agg(f.sum('Novos_casos').alias('Novos_Casos'), f.sum('Novas_Mortes').alias('Novas_Mortes'))\
                     .orderBy('Mês')

In [8]:
# df_new_month_br.limit(10).toPandas()

In [9]:
df_new_month_br.toPandas()\
  .to_csv(main_path+'aggregations/df_new_month_br.csv', sep=';', index=False, encoding='UTF-8')

#### 2.2. Novos Casos e Mortes x Dia x Brasil

In [10]:
df_new_day_br = df.withColumn('País',f.lit('Brasil'))\
                    .groupBy('Data','País')\
                    .agg(f.sum('Novos_Casos').alias('Novos_Casos'), f.sum('Novas_Mortes').alias('Novas_Mortes'))\
                     .orderBy('Data')

In [11]:
# df_new_day_br.limit(10).toPandas()

In [12]:
df_new_day_br.toPandas()\
  .to_csv(main_path+'aggregations/df_new_day_br.csv', sep=';', index=False, encoding='UTF-8')

#### 2.3. Novos Casos e Mortes x Dia Semana x Brasil

In [13]:
df_new_weekday_br = df.withColumn('País',f.lit('Brasil'))\
                        .groupBy('Dia_Semana','Dia_Semana_Nome','País')\
                        .agg(f.sum('Novos_Casos').alias('Novos_Casos'), f.sum('Novas_Mortes').alias('Novas_Mortes'))\
                        .orderBy('Dia_Semana')

In [14]:
# df_new_weekday_br.limit(10).toPandas()

In [15]:
df_new_weekday_br.toPandas()\
  .to_csv(main_path+'aggregations/df_new_weekday_br.csv', sep=';', index=False, encoding='UTF-8')

#### 2.4. Média Móvel Casos e Mortes x Dia x Brasil

In [16]:
w = Window.orderBy('Data').rowsBetween(-dias_mm, 0)
df_mvg_avg_br = df.withColumn('País',f.lit('Brasil'))\
                    .groupBy('Data','País')\
                    .agg(f.sum('Novos_Casos').alias('Novos_Casos'), f.sum('Novas_Mortes').alias('Novas_Mortes'))\
                    .withColumn('Média_Móvel_Casos', f.avg('Novos_Casos').over(w))\
                    .withColumn('Média_Móvel_Mortes', f.avg('Novas_Mortes').over(w))\
                    .withColumn('Média_Móvel_Casos', f.round('Média_Móvel_Casos',2))\
                    .withColumn('Média_Móvel_Mortes', f.round('Média_Móvel_Mortes',2))\
                    .drop('Novos_Casos','Novas_Mortes')\
                    .orderBy('Data')

In [17]:
# df_mvg_avg_br.limit(10).toPandas()

In [18]:
df_mvg_avg_br.toPandas()\
  .to_csv(main_path+'aggregations/df_mvg_avg_br.csv', sep=';', index=False, encoding='UTF-8')

#### 2.5. Letalidade x Último Dia x Brasil

In [15]:
df_let_br = df.filter(f.col('Data') == f.lit(data_max)) \
                .withColumn('País',f.lit('Brasil'))\
                .groupBy('País')\
                .agg(f.sum('Mortes_Cumul').alias('Mortes_Cumul'), f.sum('Casos_Cumul').alias('Casos_Cumul')) \
                .withColumn('Taxa_Letalidade', f.round(f.col('Mortes_Cumul') / f.col('Casos_Cumul'),4))

In [16]:
# df_let_br.limit(10).toPandas()

In [17]:
df_let_br.toPandas()\
  .to_csv(main_path+'aggregations/df_let_br.csv', sep=';', index=False, encoding='UTF-8')

### 3. Agregações Região

* Novos Casos x Dia Semana x Região
* Novas Mortes x Dia Semana x Região
* Letalidade x Último Dia x Região

#### 3.1. Novos Casos e Mortes x Dia Semana x Região

In [22]:
df_new_weekday_reg = df.withColumn('País',f.lit('Brasil'))\
                        .groupBy('Dia_Semana','Dia_Semana_Nome','Região','Região_Cod',)\
                        .agg(f.sum('Novos_Casos').alias('Novos_Casos'), f.sum('Novas_Mortes').alias('Novas_Mortes'))\
                        .orderBy('Dia_Semana','Região_Cod')\
                        .drop('Região_Cod')

In [23]:
# df_new_weekday_reg.limit(10).toPandas()

In [24]:
df_new_weekday_reg.toPandas()\
  .to_csv(main_path+'aggregations/df_new_weekday_reg.csv', sep=';', index=False, encoding='UTF-8')

#### 3.2. Letalidade x Último Dia x Região

In [25]:
df_let_reg = df.filter(f.col('Data') == f.lit(data_max)) \
                .groupBy('Região','Região_Cod',)\
                .agg(f.sum('Mortes_Cumul').alias('Mortes_Cumul'), f.sum('Casos_Cumul').alias('Casos_Cumul')) \
                .withColumn('Taxa_Letalidade', f.round(f.col('Mortes_Cumul') / f.col('Casos_Cumul'), 4))\
                .drop('Região_Cod')

In [26]:
# df_let_reg.limit(10).toPandas()

In [27]:
df_let_reg.toPandas()\
  .to_csv(main_path+'aggregations/df_let_reg.csv', sep=';', index=False, encoding='UTF-8')

### 4. Agregações UF

* Novos Casos x Mês x UF
* Novas Mortes x Mês x UF
* Novos Casos x Dia x UF
* Novas Mortes x Dia x UF
* Média Móvel Casos x Dia x UF
* Média Móvel Mortes x Dia x UF
* Letalidade x Último Dia x UF

#### 4.1. Novos Casos e Mortes x Mês x UF

In [28]:
df_new_month_uf = df.groupBy('Mês','Mês_Nome','UF')\
                     .agg(f.sum('Novos_casos').alias('Novos_Casos'), f.sum('Novas_Mortes').alias('Novas_Mortes'))\
                     .orderBy('Mês')

In [29]:
# df_new_month_uf.limit(10).toPandas()

In [30]:
df_new_month_uf.toPandas()\
  .to_csv(main_path+'aggregations/df_new_month_uf.csv', sep=';', index=False, encoding='UTF-8')

#### 4.2. Novos Casos e Mortes x Dia x UF

In [7]:
df_new_day_uf = df.groupBy('Data','UF')\
                    .agg(f.sum('Novos_casos').alias('Novos_Casos'), f.sum('Novas_Mortes').alias('Novas_Mortes'))\
                    .orderBy('Data')

In [10]:
# df_new_day_uf.limit(10).toPandas()

In [9]:
df_new_day_uf.toPandas()\
  .to_csv(main_path+'aggregations/df_new_day_uf.csv', sep=';', index=False, encoding='UTF-8')

#### 4.3. Média Móvel Casos e Mortes x Dia x UF

In [31]:
w = Window.partitionBy('UF').orderBy('Data').rowsBetween(-dias_mm, 0)
df_mvg_avg_uf = df.groupBy('Data','UF')\
                    .agg(f.sum('Novos_Casos').alias('Novos_Casos'), f.sum('Novas_Mortes').alias('Novas_Mortes'))\
                    .withColumn('Média_Móvel_Casos', f.avg('Novos_Casos').over(w))\
                    .withColumn('Média_Móvel_Mortes', f.avg('Novas_Mortes').over(w))\
                    .withColumn('Média_Móvel_Casos', f.round('Média_Móvel_Casos',2))\
                    .withColumn('Média_Móvel_Mortes', f.round('Média_Móvel_Mortes',2))\
                    .drop('Novos_Casos','Novas_Mortes')\
                    .orderBy('Data')

In [32]:
# df_mvg_avg_uf.limit(10).toPandas()

In [33]:
df_mvg_avg_uf.count()

5724

In [34]:
df_mvg_avg_uf.toPandas()\
  .to_csv(main_path+'aggregations/df_mvg_avg_uf.csv', sep=';', index=False, encoding='UTF-8')

#### 4.4. Letalidade x Último Dia x UF

In [35]:
df_let_uf = df.filter(f.col('Data') == f.lit(data_max)) \
                .groupBy('UF')\
                .agg(f.sum('Mortes_Cumul').alias('Mortes_Cumul'), f.sum('Casos_Cumul').alias('Casos_Cumul')) \
                .withColumn('Taxa_Letalidade', f.round(f.col('Mortes_Cumul') / f.col('Casos_Cumul'), 4))

In [36]:
# df_let_uf.limit(10).toPandas()

In [37]:
df_let_uf.count()

27

In [38]:
df_let_uf.toPandas()\
  .to_csv(main_path+'aggregations/df_let_uf.csv', sep=';', index=False, encoding='UTF-8')

### 5. Agregações Município

* Novos Casos x Mês x Município
* Novas Mortes x Mês x Município
* Média Móvel Casos x Dia x Município
* Média Móvel Mortes x Dia x Município
* Letalidade x Último Dia x Município

#### 5.1. Novos Casos e Mortes x Mês x Município

In [7]:
df_new_month_munic = df.groupBy('Mês','Mês_Nome','Município','Município_Cod','Região','UF')\
                        .agg(f.sum('Novos_casos').alias('Novos_Casos'), f.sum('Novas_Mortes').alias('Novas_Mortes'))\
                        .orderBy('Mês')

In [8]:
df_new_month_munic.limit(5).toPandas()

,Mês,Mês_Nome,Município,Município_Cod,Região,UF,Novos_Casos,Novas_Mortes
0,3,Março,Pombos,2611309,Nordeste,PE,0.0,0.0
1,3,Março,Bebedouro,3506102,Sudeste,SP,0.0,0.0
2,3,Março,Rialma,5218607,Centro-Oeste,GO,0.0,0.0
3,3,Março,Flora Rica,3515806,Sudeste,SP,0.0,0.0
4,3,Março,Vila Nova do Sul,4323457,Sul,RS,0.0,0.0


In [9]:
df_new_month_munic.count()

44560

In [10]:
df_new_month_munic.toPandas()\
  .to_csv(main_path+'aggregations/df_new_month_munic.csv', sep=';', index=False, encoding='UTF-8')

#### 5.2. Média Móvel Casos e Mortes x Dia x Município

In [11]:
w = Window.partitionBy('Município_Cod').orderBy('Data').rowsBetween(-dias_mm, 0)
df_mvg_avg_munic = df.groupBy('Data','Município','Município_Cod','Região','UF')\
                        .agg(f.sum('Novos_Casos').alias('Novos_Casos'), f.sum('Novas_Mortes').alias('Novas_Mortes'))\
                        .withColumn('Média_Móvel_Casos', f.avg('Novos_Casos').over(w))\
                        .withColumn('Média_Móvel_Mortes', f.avg('Novas_Mortes').over(w))\
                        .withColumn('Média_Móvel_Casos', f.round('Média_Móvel_Casos',2))\
                        .withColumn('Média_Móvel_Mortes', f.round('Média_Móvel_Mortes',2))\
                        .drop('Novos_Casos','Novas_Mortes')\
                        .orderBy('Data')

In [12]:
df_mvg_avg_munic.limit(5).toPandas()

,Data,Município,Município_Cod,Região,UF,Média_Móvel_Casos,Média_Móvel_Mortes
0,2020-03-27,Santa Terezinha,2612802,Nordeste,PE,NaN,NaN
1,2020-03-27,Rio Bonito,3304300,Sudeste,RJ,NaN,NaN
2,2020-03-27,Guarda-Mor,3128600,Sudeste,MG,NaN,NaN
3,2020-03-27,Goianinha,2404200,Nordeste,RN,NaN,NaN
4,2020-03-27,Iturama,3134400,Sudeste,MG,NaN,NaN


In [13]:
df_mvg_avg_munic.count()

1180840

In [14]:
df_mvg_avg_munic.toPandas()\
  .to_csv(main_path+'aggregations/df_mvg_avg_munic.csv', sep=';', index=False, encoding='UTF-8')

#### 5.3. Letalidade x Último Dia x Município

In [18]:
df_let_munic = df.filter(f.col('Data') == f.lit(data_max)) \
                .select('Município','Município_Cod','Casos_Cumul','Mortes_Cumul','Região','UF')\
                .withColumn('Taxa_Letalidade', f.round(f.col('Mortes_Cumul') / f.col('Casos_Cumul'), 4))

In [19]:
df_let_munic.limit(5).toPandas()

,Município,Município_Cod,Casos_Cumul,Mortes_Cumul,Região,UF,Taxa_Letalidade
0,Envira,1301506,849,2,Norte,AM,0.0024
1,Chaves,1502509,180,1,Norte,PA,0.0056
2,Amapá,1600105,774,4,Norte,AP,0.0052
3,Pedra Branca do Amapari,1600154,2753,5,Norte,AP,0.0018
4,Tartarugalzinho,1600709,1288,4,Norte,AP,0.0031


In [20]:
df_let_munic.count()

5570

In [21]:
df_let_munic.toPandas()\
  .to_csv(main_path+'aggregations/df_let_munic.csv', sep=';', index=False, encoding='UTF-8')

### 6. Classificação dos municípios pelo nível de impacto

* Municípios que oscilam entre -20% e +20% com relação à letalidade média Brasil são do grupo Impacto Médio
* Municípios com letalidade inferior à -20% da letalidade média Brasil são do grupo Baixo Impacto
* Municípios com letalidade superior à +20% da letalidade média Brasil são do grupo Alto Impacto

In [25]:
value_let_br = df_let_br.select('Taxa_Letalidade').toPandas()['Taxa_Letalidade'][0]
value_let_br

0.0292

In [26]:
df_class_munic = df_let_munic.withColumn('Taxa_Letalidade_Br', f.lit(value_let_br))\
                             .withColumn('Desvio_Br', 
                                         (f.col('Taxa_Letalidade')-f.col('Taxa_Letalidade_Br'))/f.col('Taxa_Letalidade_Br'))\
                             .withColumn('Impacto', 
                                         f.when((f.col('Desvio_Br') >= -0.2) & (f.col('Desvio_Br') <= 0.2), 'Impacto_Médio')\
                                          .otherwise(f.when(f.col('Desvio_Br') < -0.2, 'Impacto_Baixo')\
                                                      .otherwise(f.when(f.col('Desvio_Br') > 0.2, 'Impacto_Alto'))))

In [27]:
df_class_munic.limit(5).toPandas()

,Município,Município_Cod,Casos_Cumul,Mortes_Cumul,Região,UF,Taxa_Letalidade,Taxa_Letalidade_Br,Desvio_Br,Impacto
0,Envira,1301506,849,2,Norte,AM,0.0024,0.0292,-0.917808,Impacto_Baixo
1,Chaves,1502509,180,1,Norte,PA,0.0056,0.0292,-0.808219,Impacto_Baixo
2,Amapá,1600105,774,4,Norte,AP,0.0052,0.0292,-0.821918,Impacto_Baixo
3,Pedra Branca do Amapari,1600154,2753,5,Norte,AP,0.0018,0.0292,-0.938356,Impacto_Baixo
4,Tartarugalzinho,1600709,1288,4,Norte,AP,0.0031,0.0292,-0.893836,Impacto_Baixo


In [28]:
df_class_munic.count()

5570

In [29]:
df_class_munic.toPandas()\
  .to_csv(main_path+'aggregations/df_class_munic.csv', sep=';', index=False, encoding='UTF-8')